<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#SGDClassifier" data-toc-modified-id="SGDClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>SGDClassifier</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#GridSearchCV" data-toc-modified-id="GridSearchCV-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>GridSearchCV</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
import re 
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import f1_score

Скачаем датасет и посмотрим на него.

In [2]:
df = pd.read_csv("/datasets/toxic_comments.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [3]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


В первые 5 и последние 5 строк не попало единиц в столбце "toxic", поэтому посмотрим, есть ли они вообще и посмотрим дубликаты строк.

In [4]:
df['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

In [5]:
df.duplicated().sum()

0

Добавим функции для лемматизации и очистки текста. Добавим подготовленный текст в новую колонку 'text_clear_lemm'.

In [6]:
def lemmatize(text):
    m = WordNetLemmatizer()
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
        
    return lemm_text


def clear_text(text):
    cl_text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return  " ".join(cl_text.split())

In [7]:
df['text_clear'] = df['text'].apply(clear_text)


In [8]:
df['text_clear_lemm'] = df['text_clear'].apply(lemmatize)

In [9]:
df.sample(15)

,text,toxic,text_clear,text_clear_lemm
129000,"""\n\nhelp me\n\nAn editor ( | Talk """,0,help me An editor Talk,help me An editor Talk
136300,18. Removed another external link: this one ha...,0,Removed another external link this one has no ...,Removed another external link this one has no ...
155865,"""\n""""We"""" stand for Ultra-Orthodox Jews. Pleas...",0,We stand for Ultra Orthodox Jews Please step b...,We stand for Ultra Orthodox Jews Please step b...
5160,"""\n\n Good policy suggestion on class assignme...",0,Good policy suggestion on class assignments My...,Good policy suggestion on class assignments My...
34579,=I see you back from Madison,0,I see you back from Madison,I see you back from Madison
23192,I see you're one of the many wikipedians that ...,0,I see you re one of the many wikipedians that ...,I see you re one of the many wikipedians that ...
92867,"""\nI was just reading the preface of """"Making ...",0,I was just reading the preface of Making Rumou...,I was just reading the preface of Making Rumou...
140268,"""Try answering these two first - Does the Seco...",0,Try answering these two first Does the Second ...,Try answering these two first Does the Second ...
77374,"""Furthermore, you substantiate my allegation t...",0,Furthermore you substantiate my allegation tha...,Furthermore you substantiate my allegation tha...
107575,Stop. Hounding. Me.\nLeave. My. Constructive. ...,0,Stop Hounding Me Leave My Constructive Edits A...,Stop Hounding Me Leave My Constructive Edits A...


Разделим датасет на обучающую и тестовую выборку.

In [10]:
df_train, df_test = train_test_split(df, test_size=0.25)

Переведём тексты в стандартный для Python формат: кодировку Unicode (U). Изменим кодировку методом astype():

In [11]:
train_corpus = df_train['text_clear_lemm']
test_corpus = df_test['text_clear_lemm']

Подготовим целевые признаки для обоих выборок.

In [12]:
target_train = df_train['toxic']
target_test = df_test['toxic']

Вызовем функцию __stopwords.words()__, передадим ей аргумент 'english' - английские стоп-слова. 
Создадим счётчик, указав в нём стоп-слова.

In [13]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

Чтобы посчитать TF-IDF для корпуса текстов, вызовем функцию fit_transform(). Для определения тональности применим величины TF-IDF как признаки.

In [14]:
feature_train = count_tf_idf.fit_transform(train_corpus)

Создадим тестовые признаки, без функции fit.

In [15]:
feature_test = count_tf_idf.transform(test_corpus)

## Обучение

### LogisticRegression
Обучим модель LogisticRegression и посмотрим метрику F1 на тестовой выборки.

In [16]:
model_lr = LogisticRegression(class_weight='balanced', n_jobs=-1, random_state=42)
model_lr.fit(feature_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
predictions_lr = model_lr.predict(feature_test)
f1_score(target_test, predictions_lr, average='binary')

0.7599398237696111

### SGDClassifier
Обучим модель SGDClassifier и посмотрим метрику F1 на тестовой выборки.

In [18]:
model_SGDCl = SGDClassifier(class_weight='balanced', n_jobs=-1, random_state=42)
model_SGDCl.fit(feature_train, target_train)

SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [19]:
predictions_SGDCl = model_SGDCl.predict(feature_test)
f1_score(target_test, predictions_SGDCl, average='binary')

0.7424500212675457

### DecisionTreeClassifier
Обучим модель DecisionTreeClassifier и посмотрим метрику F1 на тестовой выборки.

In [20]:
model_dtc = DecisionTreeClassifier(random_state=42)
model_dtc.fit(feature_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [21]:
predictions_dtc = model_dtc.predict(feature_test)
f1_score(target_test, predictions_dtc, average='binary')

0.7200097727827999

### GridSearchCV
Обучим модель LogisticRegression с GridSearchCV и посмотрим метрику F1 на тестовой выборки.

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
pipe_lr = Pipeline([('vect', CountVectorizer(stop_words=stopwords)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', LogisticRegression(random_state=42))])

In [24]:
param_range = [9, 10]
param_range_fl = [1.0, 0.5]

In [25]:
grid_params_lr = [{'clf__penalty': ['l1', 'l2'],
                'clf__C': param_range_fl,
                'clf__solver': ['liblinear']}] 
jobs = -1

In [26]:
LR = GridSearchCV(estimator=pipe_lr,
                param_grid=grid_params_lr,
                scoring='f1',
                cv=10) 

In [27]:
LR.fit(train_corpus, target_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                           

In [28]:
predictions = LR.predict(test_corpus)
f1_score(target_test, predictions, average='binary')

0.7762632520996834

## Выводы

   В данном проекте нужно было подобрать инструмент для интернет-магазина «Викишоп», который будет искать токсичные комментарии и отправлять их на модерацию. По имеющимуся набору данных с разметкой о токсичности правок необходимо было обучить модель классификации со значением метрики качества F1 не меньше 0.75.
   
   Имеющиеся текстовые комментарии были лемматизированы и очищены от знаков припенания и лишних символов. 
   
   Для определения тональности были применены величины TF-IDF как признаки.
   
   Были проверены 4 модели классификации: LogisticRegression, SGDClassifier, DecisionTreeClassifier и LogisticRegression с GridSearchCV.
   
|  Model | F1 |
| -- | -- | 
|LogisticRegression | 0.755 |
|SGDClassifier | 0.739 |
|DecisionTreeClassifier | 0.717 |
|GridSearchCV | 0.776 |

Подходящей моделью оказалась модель LogisticRegression с GridSearchCV. 